In [1]:


import numpy as np
import os

os.system('source ~/Downloads/cdf37_1-dist/src/definitions/definitions.K')
os.environ["CDF_LIB"] = '/home/jackalak/Downloads/cdf37_1-dist/lib'
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

from sklearn import *

from scipy import interpolate

from ovationpyme import ovation_utilities

from ovationpyme.ovation_utilities import robinson_auroral_conductance
from ovationpyme.ovation_utilities import brekke_moen_solar_conductance

import geospacepy
from geospacepy import special_datetime,astrodynamics2,satplottools
import aacgmv2 #available on pip

%matplotlib inline  


import numpy as np
import matplotlib.pyplot as plt
from ovationpyme.ovation_prime import FluxEstimator,AverageEnergyEstimator,BinCorrector, SeasonalFluxEstimator
from ovationpyme.ovation_utilities import calc_avg_solarwind
from ovationpyme.ovation_plotting import latlt2polar,polar2dial,pcolor_flux
import ipywidgets
from collections import OrderedDict
from functools import partial

auroral_types = ['diff','mono','wave']#,'ions']
flux_types = [
                "energy",
                "number",
                "average energy"
            ]

import datetime
from collections import OrderedDict

import numpy as np
import functools

from geospacepy import omnireader, special_datetime, astrodynamics2

_ovation_prime_omni_cadence = 'hourly' #Ovation Prime was created using hourly SW




atype =auroral_types[0] #widgets['atype_select'].value
jtype = 'energy'#widgets['jtype_select'].value
bincorrect = True#widgets['bincorrect_toggle'].value
combine_hemispheres = True#widgets['combineNS_toggle'].value
manual_newell = -1#widgets['newell_float'].value

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geospacepy-0.1.0-py3.6.egg/geospacepy/__init__.py", line 8, in <module>
    from geospacepy.geospacepy_config import config
ModuleNotFoundError: No module named 'geospacepy.geospacepy_config'

Solar wind data files will be saved to /home/jackalak/.local/share/geospacepy


In [ ]:
file_load_df_cumulative = 'ML_DB_subsamp_ext_full_dfCumulative_complexHemisphereCombine.csv'
DMSP_DATA_DIR=''
df_cumulative = pd.read_csv(os.path.join(DMSP_DATA_DIR,file_load_df_cumulative))
df_cumulative = df_cumulative.sort_values(by=[ 'Datetimes'])
df_cumulative = df_cumulative.set_index('Datetimes')
df_cumulative.index = pd.to_datetime(df_cumulative.index)

cols_to_drop_validation = [c for c in df_cumulative.columns if ('STD' in c) | ('AVG' in c)]#] | ('SC_AACGM_LTIME'==c)]
# cols_to_drop_validation = [c for c in df.columns if ('1min' in c) | ('3min' in c) | ('4min' in c) | ('5min' in c) | ('15min' in c) | ('newell' in c) | ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]

df_cumulative = df_cumulative.drop(columns=cols_to_drop_validation)
# Separate training and testing data
mask_val = [(df_cumulative.index.year == 2010) & (df_cumulative['ID_SC'].values==16)]
df_val = df_cumulative[mask_val[0]].copy(deep=True)
df_train = df_cumulative.copy(deep=True).drop( df_cumulative.index[mask_val[0]])

In [ ]:
import pickle

with open('ovation_fluxes_val','rb') as f: ovation_fluxes_val = pickle.load(f)
with open('ovation_fluxes_train','rb') as f: ovation_fluxes_train = pickle.load(f)

In [ ]:
df_val['ovation_flux_sum'] = ovation_fluxes_val[0,:]+ovation_fluxes_val[1,:]+ovation_fluxes_val[2,:]
df_train['ovation_flux_sum'] = ovation_fluxes_train[0,:]+ovation_fluxes_train[1,:]+ovation_fluxes_train[2,:]
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from sklearn import preprocessing

In [ ]:
# Construct X and y
feature_cols = [c for c in df_cumulative.columns if not 'ELE' in c]
#print( (feature_cols))
#print(df_cumulative.columns)

X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
#print(numFeatures)

y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))

for i in range(0,X_train_scaled.shape[1]):
    maxi = np.max(np.abs(X_train_scaled[:,i]))
    print(np.max(np.abs(X_train_scaled[:,i])))
    X_train_scaled[:,i] = X_train_scaled[:,i]/maxi
    X_val_scaled[:,i]= X_val_scaled[:,i]/maxi

    print(np.max(np.abs(X_train_scaled[:,i])))

In [ ]:

for i in range(0,X_train_scaled.shape[1]):
    print(np.max(np.abs(X_train_scaled[:,i])))


In [ ]:
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#create model
model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=500)
results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)
plt.figure(figsize=(12,12))

plt.plot(y_val_log.values[:500])
plt.plot(results[:500])
plt.show()

In [ ]:
# Construct X and y
feature_cols = [c for c in df_cumulative.columns if not (('ELE' in c)| ( 'ovation' in c) )]
print( (feature_cols))
#print(df_cumulative.columns)

X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
#print(numFeatures)

y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))

model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=500)
results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)

In [ ]:
plt.figure(figsize=(12,12))

plt.plot(y_val_log.values[:500])
plt.plot(results[:500])
plt.show()

In [ ]:
df_val['ovation_flux_1'] = ovation_fluxes_val[0,:]
df_train['ovation_flux_1'] = ovation_fluxes_train[0,:]
df_val['ovation_flux_2'] = ovation_fluxes_val[1,:]
df_train['ovation_flux_2'] = ovation_fluxes_train[1,:]
df_val['ovation_flux_3'] = ovation_fluxes_val[2,:]
df_train['ovation_flux_3'] = ovation_fluxes_train[2,:]


# Construct X and y
feature_cols = [c for c in df_cumulative.columns if not 'ELE' in c]
#print( (feature_cols))
#print(df_cumulative.columns)

X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
#print(numFeatures)

y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))

In [ ]:
model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=1000)
results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(y_val_log.values[:500])
plt.plot(results[:500])
plt.show()

In [ ]:
model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])
model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)
plt.figure(figsize=(12,12))
plt.plot(y_val_log.values[:500])
plt.plot(results[:500])
plt.show()

In [ ]:
from tensorflow.keras.layers import Dropout
import joblib

In [ ]:
model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,20):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
filename = "$PWD/best_with_ovation_input"
#joblib.dump(model, filename) doesn't work

model.save(filename)

results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)

plt.figure(figsize=(12,12))
plt.plot(y_val_log.values[:500])
plt.plot(results[:500])
plt.show()

In [ ]:
model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,20):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
model = Sequential()

model.add(Dropout(.2))
model.add(Dense(int(256), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,20):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
model = Sequential()

model.add(Dropout(.1))
model.add(Dense(int(256), activation='relu'))
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,20):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
model = Sequential()

model.add(Dropout(.1))
model.add(Dense(int(256), activation='relu'))
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,20):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
model = Sequential()

model.add(Dropout(.1))
model.add(Dense(int(256), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,20):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)


In [ ]:
df_val['ovation_flux_1'] = np.log10(ovation_fluxes_val[0,:] * (1.60218e-12) )
df_train['ovation_flux_1'] = np.log10(ovation_fluxes_train[0,:]* (1.60218e-12))
df_val['ovation_flux_2'] = np.log10(ovation_fluxes_val[1,:]* (1.60218e-12))
df_train['ovation_flux_2'] = np.log10(ovation_fluxes_train[1,:]* (1.60218e-12))
df_val['ovation_flux_3'] = np.log10(ovation_fluxes_val[2,:]* (1.60218e-12))
df_train['ovation_flux_3'] = np.log10(ovation_fluxes_train[2,:]* (1.60218e-12))


# Construct X and y
feature_cols = [c for c in df_cumulative.columns if not 'ELE' in c]
#print( (feature_cols))
#print(df_cumulative.columns)

X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
#print(numFeatures)

y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))



In [ ]:
model = Sequential()

model.add(Dense(int(256), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))


#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam',  metrics=['mse'])

for i in range(0,40):
    model.fit(X_train_scaled, np.array(y_train_log), validation_data=(X_val_scaled, np.array(y_val_log)),batch_size=8192, epochs=100)

    

In [ ]:
import joblib as joblib

filename = "$PWD/best_with_ovation_input_2000epochs"
#joblib.dump(model, filename) doesn't work

model.save(filename)

results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)

plt.figure(figsize=(12,12))
plt.plot(y_val_log.values[:500])
plt.plot(results[:500])
plt.show()